In [3]:
import sys
sys.path.append('..')

In [4]:
import lib
from lib import utils

import pandas as pd
import numpy as np
from qgrid import show_grid

from lib.obo import read_ontology
from lib.classification.elastic import search_term, collapse_matches


import data
from elasticsearch import Elasticsearch

es = Elasticsearch()

In [5]:
larisa_series = data.larisa_series()

larisa_ids = list(map(int, larisa_series.index.unique()))
larisa_vd = pd.DataFrame(dict(classes=larisa_series[['doid']]
                         .reset_index()
                         .groupby('id')['doid']
                         .apply(lambda x: list(set(x)))))
larisa_vd[:7]

,classes
id,
2817,[DOID:0060108]
3292,[DOID:5520]
3467,[DOID:3969]
3744,[DOID:1612]
4107,[DOID:9256]
4183,"[DOID:0050589, DOID:9256, DOID:0050860]"
4290,"[DOID:3068, DOID:3181, DOID:3069]"


In [12]:
def names(ids):
    return [ontology.meta[item_id].name for item_id in ids]

In [14]:
tissues_assigned_fixed = pd.read_csv('../data/geo-annotation/larisa-tissues-assigned-fixed.csv')
tissues_assigned_fixed.head()

,tissue,tissue_assigned,oid_assigned
0,breast sample,breast,BTO:0000149
1,Mammary Gland Cell,mammary gland,BTO:0000817
2,CD4+ T cells,NaN,NaN
3,epithelial cells,epithelial cell,BTO:0000414
4,stroma cells,prostate gland stromal cell,BTO:0003972


In [15]:
series_tissues = ( 
    pd.merge(larisa_series, 
             tissues_assigned_fixed, 
             left_on='tissue', 
             right_on='tissue', 
             how='left')
    [['platform', 'series', 'tissue', 'tissue_assigned', 'oid_assigned']]
    .reset_index()
    .drop('index', axis=1)
)

series_tissues.head()


,platform,series,tissue,tissue_assigned,oid_assigned
0,GPL570,GSE65194,breast sample,breast,BTO:0000149
1,GPL570,GSE65194,breast sample,breast,BTO:0000149
2,GPL570,GSE65194,breast sample,breast,BTO:0000149
3,GPL570,GSE51450,breast sample,breast,BTO:0000149
4,GPL570,GSE51450,breast sample,breast,BTO:0000149


In [16]:
good_series = (
    series_tissues
    [['series', 'oid_assigned']]
    .groupby('series')
    .oid_assigned
    .apply(lambda oids: not any(pd.isnull(oid) for oid in oids))
)
good_series = good_series[good_series]

In [17]:
series_tissues = series_tissues[series_tissues.series.isin(good_series.index)].drop_duplicates()

In [18]:
series_tissues['series'].unique().shape

(288,)

In [19]:
( 
    series_tissues
    [['series', 'oid_assigned']]
    .groupby('series')
    .oid_assigned
    .apply(lambda d: list(set(d)))
    .to_frame()
    .rename(columns={
            'oid_assigned': 'classes'
        })
    .to_pickle('../data/geo-annotation/val-larisa-tissues.py3.pickle')
)